<!--
#  Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#    Licensed under the Apache License, Version 2.0 (the "License").
#    You may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.
-->

# Creating FSX Volume for Teams to use
## Content
1. Admin Operations
    1. [Parameters](#Parameters)
    2. [Cleanup](#Cleanup)
    3. [Creating the EBS Volume](#Creating-the-EBS-Volume)
    4. [Creating the K8 Volume](#Creating-the-K8-Volume)
2. User Operations
    1. [Creating the K8 Volume Claim](#Creating-the-K8-Volume-Claim)
    2. [Creating the Profile with the required AZ](#Creating-the-Profile-with-the-required-AZ)
    3. [Running the container](#Running-the-container)
---
---





In [ ]:
from aws_orbit_sdk import controller
from aws_orbit_sdk.magics.orbit import OrbitWorkbenchMagics 
import json
import boto3
from aws_orbit_sdk.common import get_workspace

In [ ]:
# we will need the team kms key from workspace
workspace = get_workspace()
team_kms_key = workspace['TeamKmsKeyArn']
image = workspace['BaseImageAddress']

In [ ]:
%cd fsx

In [ ]:
env_name = %env AWS_ORBIT_ENV
team_name = %env AWS_ORBIT_TEAM_SPACE
(env_name,team_name)

## Parameters

In [ ]:
pvc_name = f'lustre-1200g-{team_name}'

volume_size = "1200Gi"

## Creating the FSX FileSystem and K8 PVC

In [ ]:
with open("pvc.yaml", "w") as file:
    file.write("""
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: {pvc_name}
  namespace: {team_name}
  labels:
    orbit/node-type: ec2
    orbit/attach-security-group: "yes"    
spec:
  accessModes:
    - ReadWriteMany
  storageClassName: fsx-lustre-{team_name}-fast-fs-lustre
  resources:
    requests:
      storage: {volume_size} 
    """.format(team_name=team_name,pvc_name=pvc_name,volume_size=volume_size,image=image)
)




In [ ]:
!cat pvc.yaml

In [ ]:
!kubectl apply -f pvc.yaml

In [ ]:
with open("chmodpod.yaml", "w") as file:
    file.write("""
apiVersion: batch/v1
kind: Job
metadata:
  name: chmod-fsx-pod
  namespace: {team_name}
  labels:
    app: test
    orbit/node-type: ec2
    orbit/attach-security-group: "yes"
spec:
  template:
    spec:
      nodeSelector:
        orbit/usage: teams
        orbit/node-type: ec2
      containers:
      - name: ownership-change
        image: {image}
        securityContext:
            runAsUser: 0
        command: ["sh", "-c", "sudo chmod 777 /data"]
        volumeMounts:
        - name: persistent-storage
          mountPath: /data
      restartPolicy: Never
      volumes:
      - name: persistent-storage
        persistentVolumeClaim:
          claimName: {pvc_name}
    """.format(team_name=team_name,pvc_name=pvc_name,volume_size=volume_size,image=image)
)

In [ ]:
!cat chmodpod.yaml

In [ ]:
# Need to wait sometime for FS to get created
!sleep 8m

In [ ]:
!kubectl apply -f chmodpod.yaml

## User Section

In [ ]:
!orbit list profile --env $env_name --team $team_name

## Running the container

In [ ]:
run = {    
      "tasks":  [
            {
                  "notebookName": "test-fsx.ipynb",
                  "sourcePath": "shared/samples/notebooks/M-Admin/fsx",
                  "targetPath": "shared/regression/notebooks/M-Admin/fsx",
                  "params": {
                        "test" : "1"
                  }      
            }
      ],
      "compute": {
          "container" : {
              "p_concurrent": "1"
          },
          "node_type": "ec2",
          "profile": "nano",
          "labels": {
            "my-jobid": "1"
          },          
          "volumes": [{"name": "fsx-volume", "persistentVolumeClaim": {"claimName": pvc_name}}],
          "volume_mounts": [{"mountPath": "/fsx", "name": "fsx-volume"}]
      }
}

with open("run.json", 'w') as f:
    json.dump(run, f)


In [ ]:
!cat run.json

In [ ]:
%%time

!orbit run notebook --env $env_name --team $team_name --user testing --wait --tail-logs run.json --delay 60 --max-attempts 15


In [ ]:
output = !ls ~/shared/regression/notebooks/M-Admin/fsx/test-fsx
print(output)
assert('error' not in str(output))


In [ ]:
output[0]

## Cleanup

In [ ]:
# Using our label to delete the job
!kubectl delete job -l my-jobid=1